In [8]:
# Import the required packeages
import datetime
import os
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from dotenv import load_dotenv 
from pathlib import Path

In [9]:
# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [10]:
# Obtain the envrionment variables from .env file
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [11]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [12]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

In [13]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

### Download data from modelindex api

In [14]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

[{'Idx': 0, 'Uri': 'http://opcfoundation.org/UA/'},
 {'Idx': 1, 'Uri': 'http://prediktor.no/apis/ua/'},
 {'Idx': 2, 'Uri': 'urn:prediktor:UIDEV-W2022-04:Scatec'},
 {'Idx': 3, 'Uri': 'http://scatecsolar.com/EG-AS'},
 {'Idx': 4, 'Uri': 'http://scatecsolar.com/Enterprise'},
 {'Idx': 5, 'Uri': 'http://scatecsolar.com/JO-GL'},
 {'Idx': 6, 'Uri': 'http://prediktor.no/PVTypes/'},
 {'Idx': 7, 'Uri': 'http://powerview.com/enterprise'}]

In [15]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

,Id,Name,BrowseName,Props,Vars
0,6:0:1061,EquipmentEventType,EquipmentEventType,[],[]
1,6:0:1128,EnergyAndPowerMeterEventType,EnergyAndPowerMeterEventType,[],[]
2,6:0:1263,EnergyAndPowerMeterCommLossEventType,EnergyAndPowerMeterCommLossEventType,[],[]
3,6:0:1266,EnergyAndPowerMeterErrorEventType,EnergyAndPowerMeterErrorEventType,[],[]
4,6:0:1269,EnergyAndPowerMeterWarningEventType,EnergyAndPowerMeterWarningEventType,[],[]
...,...,...,...,...,...
217,6:0:1013,GridType,GridType,[],[]
218,6:0:1011,SectionType,SectionType,[],[]
219,6:0:1009,SiteType,SiteType,[],[]
220,6:0:1010,SubSiteType,SubSiteType,[],[]


In [16]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client)

In [17]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

,Id,Name
0,6:0:1061,EquipmentEventType
1,6:0:1128,EnergyAndPowerMeterEventType
2,6:0:1263,EnergyAndPowerMeterCommLossEventType
3,6:0:1266,EnergyAndPowerMeterErrorEventType
4,6:0:1269,EnergyAndPowerMeterWarningEventType
...,...,...
106,6:0:1013,GridType
107,6:0:1011,SectionType
108,6:0:1009,SiteType
109,6:0:1010,SubSiteType


In [18]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

'6:0:1009'

In [19]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

['EG-AS', 'JO-GL']

In [20]:
# Analytics helper
sites.variables_as_dataframe()

,Id,Type,Name,VariableId,VariableName,VariableIdSplit
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Alarms.CommLossPlantDevice,CommLossPlantDevice,{'Id': 'Enterprise.EG-AS.Alarms.CommLossPlantD...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Signals.PPC.IsCurtailment,PPC.IsCurtailment,{'Id': 'Enterprise.EG-AS.Signals.PPC.IsCurtail...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Signals.State.IsDay,State.IsDay,"{'Id': 'Enterprise.EG-AS.Signals.State.IsDay',..."
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Parameters.ContractDuration,ContractDuration,{'Id': 'Enterprise.EG-AS.Parameters.ContractDu...
0,3:1:Enterprise.EG-AS,6:0:1009,EG-AS,3:1:Enterprise.EG-AS.Parameters.RegionKey,RegionKey,{'Id': 'Enterprise.EG-AS.Parameters.RegionKey'...
...,...,...,...,...,...,...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.PPC.SetpointActiv...,PPC.SetpointActivePower,{'Id': 'Enterprise.JO-GL.Signals.PPC.SetpointA...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationDiffuseHorizontal,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationHorizontal,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...
1,5:1:Enterprise.JO-GL,6:0:1009,JO-GL,5:1:Enterprise.JO-GL.Signals.Weather.Irradiati...,Weather.IrradiationInCline,{'Id': 'Enterprise.JO-GL.Signals.Weather.Irrad...


In [21]:
sites.list_of_ids()

['5:1:Enterprise.JO-GL', '3:1:Enterprise.EG-AS']

In [22]:
# Selecting the single site
site_id = sites.list_of_ids()[1]
site_id

'3:1:Enterprise.EG-AS'

In [23]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

,Id,Name,Type,Props,Vars
0,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH01,EG-AS-TS01-I01-SM01-CH01,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '1'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
1,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH02,EG-AS-TS01-I01-SM01-CH02,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '2'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH03,EG-AS-TS01-I01-SM01-CH03,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '3'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
3,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH04,EG-AS-TS01-I01-SM01-CH04,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '4'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
4,3:1:Enterprise.EG-AS.S1.Z5.TS01.I01.SM01.CH05,EG-AS-TS01-I01-SM01-CH05,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '5'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
...,...,...,...,...,...
2933,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH05,EG-AS-TS11-I22-SM13-CH05,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '5'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2934,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH06,EG-AS-TS11-I22-SM13-CH06,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '6'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2935,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH07,EG-AS-TS11-I22-SM13-CH07,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '7'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."
2936,3:1:Enterprise.EG-AS.S1.Z1.TS11.I22.SM13.CH08,EG-AS-TS11-I22-SM13-CH08,StringSetType,"[{'DisplayName': 'ChannelNo', 'Value': '8'}, {...","[{'DisplayName': 'StringDisconnected', 'Id': '..."


In [24]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

,Id,Name,Type,VariableId,VariableName,VariableIdSplit
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,TrackerOutOfPos,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasHighSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasMediumSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,State.HasLowSeverityAlarm,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
0,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001,EG-AS-TR-TB01.TR001,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001....,CommLoss,{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR...
...,...,...,...,...,...,...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,TrackingLimitWestAngle,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,MotorPressure,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,Category,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...
5871,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178,EG-AS-TR-TB11.TR178,TrackerType,3:1:Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR178....,StateCode,{'Id': 'Enterprise.EG-AS.S1.MMSTB11.TR-TB11.TR...


### Download data from the opc ua api

In [25]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"])
)
live_value

[{'Id': 'Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Signals.AngleMeasured',
  'Namespace': 3,
  'IdType': 1,
  'Timestamp': '2024-02-22T09:26:12.4136116Z',
  'Value': -3.7381437,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None}]

In [26]:
# Historic value data of trackers, 1 days worth of data 30 days ago
one_day_historic_tracker_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured"]),
)
one_day_historic_tracker_data

C:\Users\meena.bana\OneDrive - TGS Prediktor AS\Dokumenter\git_repos\pyPrediktorMapClient\src\pyprediktormapclient\opc_ua.py:375: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Double' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_result.at[i, "Value.Type"] = self._get_value_type(int(row["Value.Type"])).get(


,Timestamp,ValueType,Value,StatusCode,StatusSymbol,Id,HistoryReadResults.NodeId.Id,Namespace
0,2024-01-23T10:26:41Z,Double,23.492256,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
1,2024-01-23T11:26:41Z,Double,40.113758,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
2,2024-01-23T12:26:41Z,Double,45.110686,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
3,2024-01-23T13:26:41Z,Double,29.932867,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
4,2024-01-23T14:26:41Z,Double,9.147130,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
5,2024-01-23T15:26:41Z,Double,11.204942,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
6,2024-01-23T16:26:41Z,Double,11.146812,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
7,2024-01-23T17:26:41Z,Double,11.109765,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
8,2024-01-23T18:26:41Z,Double,11.102463,1083506689,UncertainSubNormal,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3
9,2024-01-23T19:26:41Z,Double,11.070352,1,Good,1,Enterprise.EG-AS.S1.MMSTB01.TR-TB01.TR001.Sign...,3


##### Ory Authentication

In [27]:
import requests
import logging
from pydantic import AnyUrl, validate_call
from typing import Literal

In [28]:
from pydantic import BaseModel, AnyUrl, validate_call, AwareDatetime, field_validator
from pyprediktormapclient.shared import request_from_api
import datetime
import json
import re

In [29]:
class Ory_Login_Structure(BaseModel):
    method: str
    identifier: str
    password: str

In [30]:
class Token(BaseModel):
    access_token: str
    expires_at: AwareDatetime = None
    
    @field_validator('expires_at', mode='before')
    def remove_nanoseconds(cls, v):
        if v is None:
            return v
        elif re.match(r"(\d\d\d\d)-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).\d+(\S+)", v):
            return datetime.datetime.strptime(f"{v[:-11]}.+00:00", "%Y-%m-%dT%H:%M:%S.%z")
        return v

In [31]:
rest_url = ory_url
headers = {"Content-Type": "application/json"}

In [32]:
def request_from_api(
    rest_url: AnyUrl,
    method: Literal["GET", "POST"],
    endpoint: str,
    data: str = None,
    params: dict = None,
    headers: dict = None,
    extended_timeout: bool = False,
) -> str:
    """Function to perform the request to the ModelIndex server

    Args:
        rest_url (str): The URL with trailing shash
        method (str): "GET" or "POST"
        endpoint (str): The last part of the url (without the leading slash)
        data (str): defaults to None but can contain the data to send to the endpoint
        headers (str): default to None but can contain the headers og the request
    Returns:
        JSON: The result if successfull
    """
    request_timeout = (3, 300 if extended_timeout else 27)
    combined_url = f"{rest_url}{endpoint}"

    result = None
    if method == "GET":
        result = requests.get(combined_url, timeout=request_timeout, params=params, headers=headers)

    if method == "POST":
        result = requests.post(
            combined_url, data=data, headers=headers, timeout=request_timeout, params=params
        )
    
    result.raise_for_status()

    if 'application/json' in result.headers.get('Content-Type', ''):
        return result.json()

    else:
        logging.warning(f"Non-JSON response received from {combined_url}")
        return {"error": "Non-JSON response", "content": result.text}

In [33]:
content = request_from_api(
    rest_url=rest_url,
    method="GET",
    endpoint="self-service/login/api",
    headers=headers,
    extended_timeout=True,
)

In [34]:
def get_login_id(content):
    """Request login token from Ory"""
    print("Getting login ID...")
    
    print(f"Response for login ID: {content}")

    if "error" in content:
        # Handle the error appropriately
        raise RuntimeError(content["error"])

    if content.get("Success") is False or not isinstance(content.get("id"), str):
        error_message = content.get("ErrorMessage", "Unknown error occurred during login.")
        raise RuntimeError(error_message)

    id = content.get("id")
    return id

In [35]:
id = get_login_id(content)
id

Getting login ID...
Response for login ID: {'id': 'dca8c4f3-2d41-4869-8a27-7480007d8064', 'oauth2_login_challenge': None, 'type': 'api', 'expires_at': '2024-02-22T09:37:22.670330564Z', 'issued_at': '2024-02-22T09:27:22.670330564Z', 'request_url': 'https://kratos.qa.powerview.io/self-service/login/api', 'ui': {'action': 'https://kratos.qa.powerview.io/self-service/login?flow=dca8c4f3-2d41-4869-8a27-7480007d8064', 'method': 'POST', 'nodes': [{'type': 'input', 'group': 'default', 'attributes': {'name': 'csrf_token', 'type': 'hidden', 'value': '', 'required': True, 'disabled': False, 'node_type': 'input'}, 'messages': [], 'meta': {}}, {'type': 'input', 'group': 'default', 'attributes': {'name': 'identifier', 'type': 'text', 'value': '', 'required': True, 'disabled': False, 'node_type': 'input'}, 'messages': [], 'meta': {'label': {'id': 1070004, 'text': 'ID', 'type': 'info'}}}, {'type': 'input', 'group': 'password', 'attributes': {'name': 'password', 'type': 'password', 'required': True, 'a

'dca8c4f3-2d41-4869-8a27-7480007d8064'

In [31]:
params = {"flow": id}
body = (Ory_Login_Structure(method="password", identifier=username, password=password).model_dump())

In [32]:
content = request_from_api(
    rest_url=rest_url,
    method="POST",
    endpoint="self-service/login",
    data=json.dumps(body),
    params=params,
    headers=headers,
    extended_timeout=True,
)

In [33]:
def get_login_token(content) -> None:
    """Request login token from Ory"""
    print("Getting login token...")
    
    print(f"Response for login token: {content}")

    if content.get("Success") is False:
        raise RuntimeError(content.get("ErrorMessage"))
    
    # Return if no content from server
    if not isinstance(content.get("session_token"), str):
        raise RuntimeError(content.get("ErrorMessage"))
    token = Token(access_token=content.get("session_token"))

    # Check if token has expiry date, save it if it does
    if isinstance(content.get("session").get("expires_at"), str):
        # String returned from ory has to many chars in microsec. Remove them
        #from_string = content.get("session").get("expires_at")
        #date_object = datetime.datetime.strptime(f"{from_string[:-11]}.+00:00", "%Y-%m-%dT%H:%M:%S.%z")
        try:
            token = Token(access_token=token.access_token, expires_at=content.get("session").get("expires_at"))
        except Exception:
            # If string returned from Ory cant be parsed, still should be possible to use Ory,
            #  might be a setting in Ory to not return expiry date
            token = Token(access_token=token.access_token)

In [34]:
token = get_login_token(content)
token

Getting login token...
Response for login token: {'session_token': 'ory_st_iFqIcvXz0Mq7m2GaPdeCK54xMb36R3zx', 'session': {'id': '738f383c-5272-4ec9-a2fa-60ed56156f21', 'active': True, 'expires_at': '2024-02-22T11:44:07.555758254Z', 'authenticated_at': '2024-02-21T11:44:07.555758254Z', 'authenticator_assurance_level': 'aal1', 'authentication_methods': [{'method': 'password', 'aal': 'aal1', 'completed_at': '2024-02-21T11:44:07.555754507Z'}], 'issued_at': '2024-02-21T11:44:07.555758254Z', 'identity': {'id': 'a97539e1-77fb-4f3c-b8c2-8aeb5aacc605', 'schema_id': 'default', 'schema_url': 'https://kratos.qa.powerview.io/schemas/ZGVmYXVsdA', 'state': 'active', 'state_changed_at': '2024-02-20T12:27:28.367035Z', 'traits': {'email': 'api_test@prediktor.com'}, 'verifiable_addresses': [{'id': 'de3233ee-c2d7-4253-b4c6-ffada0aaf858', 'value': 'api_test@prediktor.com', 'verified': False, 'via': 'email', 'status': 'pending', 'created_at': '2024-02-20T12:27:28.375424Z', 'updated_at': '2024-02-20T12:27:28

In [35]:
def check_if_token_has_expired() -> bool:
    """Check if token has expired
    """
    if token is None or token.expires_at is None:
        return True

    return datetime.datetime.utcnow() > token.expires_at

In [36]:
check_if_token_has_expired()

True

In [37]:
model_data = ModelIndex(url=model_index_url, auth_client=token)
model_data

In [38]:
def get_namespace_array() -> str:
    """Get the namespace array

    Returns:
        str: the JSON returned from the server
    """
    content = request_from_api(model_index_url, "GET", "query/namespace-array")

    return content

In [39]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

{'error': 'Non-JSON response',
 'content': '<!DOCTYPE html>\n<html\n  lang="en"\n  style="background-color: transparent"\n>\n  <head>\n    <meta charset="utf-8" />\n    <link rel="icon" type="image/png" sizes="32x32" href="/assets/pview-favicon-32x32-tqwgDONr.png">\n    <link rel="icon" type="image/png" sizes="16x16" href="/assets/pview-favicon-16x16-RlZ54WMk.png">\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <meta name="theme-color" content="#000000" />\n    <meta\n      name="description"\n      content="Web site created using create-react-app"\n    />\n    <base href="/">\n    <link rel="apple-touch-icon" href="/assets/pview-apple-touch-icon-73rAYVAM.png" />\n\t\t\t\t<!--\n\t\t\t\t\tmanifest.json provides metadata used when your web app is installed on a\n\t\t\t\t\tuser\'s mobile device or desktop. See https://developers.google.com/web/fundamentals/web-app-manifest/\n\t\t\t\t-->\n\t\t\t\t<link rel="manifest" href="/assets/manifest-H8ytzDcP.json" /

In [40]:
def request_new_ory_token() :
    """Request Ory token"""
    print("Requesting new ORY token...")
    get_login_id()
    get_login_token()
    print("New ORY token requested.")